In [1]:
import pandas as pd
import numpy as np
import datetime
import time
from calendar import monthrange
from pprint import pprint
from pandas.tseries.offsets import BDay

In [2]:
df = pd.read_csv("../../../Data/vw_Incident.csv",encoding='latin-1',low_memory=False)

In [3]:
from sklearn.utils import resample
df = resample(df, n_samples=10, random_state=1234)

In [40]:
pprint(cutoffs)

[datetime.datetime(2017, 1, 1, 0, 0),
 Timestamp('2017-01-27 17:00:00'),
 Timestamp('2017-01-30 23:00:00'),
 Timestamp('2017-02-02 23:00:00'),
 Timestamp('2017-02-01 08:00:00'),
 Timestamp('2017-02-24 17:00:00'),
 Timestamp('2017-02-27 23:00:00'),
 Timestamp('2017-03-02 23:00:00'),
 Timestamp('2017-03-01 08:00:00'),
 Timestamp('2017-03-29 17:00:00'),
 Timestamp('2017-03-30 23:00:00'),
 Timestamp('2017-04-03 23:00:00'),
 Timestamp('2017-04-01 08:00:00'),
 Timestamp('2017-04-26 17:00:00'),
 Timestamp('2017-04-27 23:00:00'),
 Timestamp('2017-05-01 23:00:00'),
 Timestamp('2017-04-29 08:00:00'),
 Timestamp('2017-05-29 17:00:00'),
 Timestamp('2017-05-30 23:00:00'),
 Timestamp('2017-06-02 23:00:00'),
 Timestamp('2017-06-01 08:00:00'),
 Timestamp('2017-06-28 17:00:00'),
 Timestamp('2017-06-29 23:00:00'),
 Timestamp('2017-07-03 23:00:00'),
 Timestamp('2017-07-01 08:00:00'),
 Timestamp('2017-07-27 17:00:00'),
 Timestamp('2017-07-28 23:00:00'),
 Timestamp('2017-08-02 23:00:00'),
 Timestamp('2017-

In [32]:
def get_last_bdays_months():
    last_bdays = pd.date_range("2017.01.01", periods=11, freq='BM')
    last_bdays_offset = []
    for last_bday in last_bdays:
        last_bdays_offset.append(last_bday + pd.DateOffset(days=1,hours=8))
    return last_bdays_offset

def get_cutoffs(last_bdays):
    start_of_the_year = pd.datetime(2017,1,1)
    last_bdays_offset = [start_of_the_year]
    for last_bday in last_bdays:
        end_fourth_last = (last_bday - BDay(1) - BDay(1) - BDay(1) - BDay(1)).normalize() +pd.DateOffset(hours=17)
        last_bdays_offset.append(end_fourth_last)
        end_third_last = (last_bday - BDay(1) - BDay(1) - BDay(1)).normalize() +pd.DateOffset(hours=23)
        last_bdays_offset.append(end_third_last)
        end_second_last = (last_bday - BDay(1) - BDay(1)).normalize() +pd.DateOffset(hours=23)
        last_bdays_offset.append(end_second_last)
        last_bdays_offset.append(last_bday)
    return last_bdays_offset
last_bdays_months = get_last_bdays_months()
cutoffs = get_cutoffs(last_bdays_months)

def get_seconds_until_end_of_day(date, cutoffs):
    for i in range(len(cutoffs)):    
        if date >= cutoffs[i] and date < cutoffs[i+1]:
            hour = cutoffs[i+1].hour
            minute = cutoffs[i+1].minute
            second = cutoffs[i+1].second
            cutoff_in_seconds = hour*60*60 + minute*60 + second

            hour = date.hour
            minute = date.minute
            second = date.second
            date_in_seconds = hour*60*60 + minute*60 + second
            
            if date.weekday() < 5:
                print(date.weekday())
                print((date + pd.DateOffset(days=1)).weekday())
                if cutoffs[i+1].time() > date.time():
                    return cutoff_in_seconds - date_in_seconds
                elif (date + pd.DateOffset(days=1)).weekday() <5:
                    
                    return cutoff_in_seconds + 24*60*60 - date_in_seconds
                else:
                    print("here")
                    return cutoff_in_seconds + (24*60*60)*3 - date_in_seconds
            else:
                if date.date() == cutoffs[i+1].date():
                    if cutoffs[i+1].hour == 23:
                        return cutoff_in_seconds - date_in_seconds
                    else:
                        print("yes")
                        return cutoff_in_seconds +24*60*60 - date_in_seconds
                else:
                    print("weekend, day number = ", date.weekday())
                    if date.weekday() == 5:
                        return (24*60*60)*2 - date_in_seconds + cutoff_in_seconds
                    else:
                        return (24*60*60) - date_in_seconds + cutoff_in_seconds
# pprint(cutoffs)
date_hour = 18
date = pd.datetime(2017, 3, 20, date_hour)
print("alg sec left =", get_seconds_until_end_of_day(date, cutoffs))

cutoff_hour = 17

# time_left = cutoff_hour+24 - date_hour # 20th Mar 16
# time_left = cutoff_hour - date_hour # 20th Mar 16
# time_left = cutoff_hour + 24- date_hour # 19th Mar 16
# time_left = cutoff_hour +24 + 24- date_hour # 18th Mar 18
# time_left = cutoff_hour +24 + 24 + 24- date_hour # 17th Mar 18
# time_left = cutoff_hour - date_hour # 17th Mar 16
# time_left = cutoff_hour +24- date_hour # 16th Mar 18
# time_left = cutoff_hour - date_hour # 1th Mar
# time_left = cutoff_hour +24- date_hour # 28th Feb
# time_left = cutoff_hour - date_hour # 27th Feb
# time_left = cutoff_hour +24 - date_hour # 26th Feb
# time_left = cutoff_hour +24+ 24 - date_hour # 25th Feb
# time_left = cutoff_hour - date_hour # 24th Feb
# time_left = 24 - date_hour + cutoff_hour # sunday before 17
# time_left = 24 - date_hour + 24 + cutoff_hour # saturday 
# print(cutoff_hour + 24 - date_hour)## weekday after 17
# print(cutoff_hour - date_hour)## weekday before 17

print("seconds left =", time_left*60*60)
print("hours left =", time_left)

0
1
alg sec left = 82800
seconds left = 82800
hours left = 23


In [4]:
def days_of_the_month_cutoffs(df, last_bdays, last_3_bdays):
    # generate the daily cutoffs for every business day in the month
    start_month = df["Created_On"].min().month
    end_month = df["Created_On"].max().month
    print(end_month - start_month)
last_bdays_months = get_last_bdays_months()
last_3_bdays = get_last_3_bdays_cutoffs(last_bdays_months)
days_of_the_month_cutoffs(df, last_bdays_months, last_3_bdays)       

TypeError: int() argument must be a string, a bytes-like object or a number, not 'type'

In [ ]:
def get_end_of_day_seconds_left(date, last_bdays):
    last_33_hour_push = 33*60*60
    for i in range(len(last_bdays_offset)):    
        if date >= last_bdays_offset[i] and date < last_bdays_offset[i+1]:
            seconds_left = (last_bdays_offset[i+1] - date).seconds
            days_left = (last_bdays_offset[i+1] - date).days
            total_seconds_left = seconds_left + days_left*24*60*60  
            if total_seconds_left < last_33_hour_push:
                return total_seconds_left
            elif total_seconds_left < last_33_hour_push + 24 + 24:
                if date.day is sunday or saturday or (date.day is friday and date.time is >11pm):
                    return total_seconds_left
                elif date.time > 11pm:
                    return total_seconds_left
                else:
                    return 11pm - date.time
            elif last_bdays_offset[i+1] - 2*day == date.day
                if date.time > 11pm:
                    return 
                else:
                    return 11pm - date.time
                
            
            
        
        
        

In [ ]:
# def get_end_of_day_seconds_left(date, last_bdays_offset, last_2_bdays):
#     last_33_hour_push = 33*60*60
#     last_2_bdays = last_33_hour_push + (23+24+5)*60*60 
#     last_2_bdays_end_of_day = 11 *60*60
#     normal_end_of_day = 5 *60*60
#     time_of_day_in_seconds = date.hour*60*60 + date.minute*60 + date.second
#     print("time_of_day_in_seconds = ", time_of_day_in_seconds)
    
#     for i in range(len(last_bdays_offset)):       
# #         if date >= last_bdays_offset[i] and date <last_bdays_offset[i+1]:
            
#             seconds_left = (last_bdays_offset[i+1] - date).seconds
#             days_left = (last_bdays_offset[i+1] - date).days
#             total_seconds_left = seconds_left + days_left*24*60*60    
#             print("time left = ", last_bdays_offset[i+1] - date)
#             print("seconds_left = ", seconds_left)
#             print("days_left = ", days_left)
#             print("total_seconds_left = ", total_seconds_left)
#             # if it's during the last 33 hours (time between the last 23 hours cutoff and end of previous business day)
            
# #             if total_seconds_left <= last_33_hour_push:
# #                 print("last_33_hour_push")
# #                 return total_seconds_left
# #             elif date == last_bdays_offset[i+1] - BDay or date == last_bdays_offset[i+1] - BDay -BDay:
# #                 if date.time < 11pm:
# #                     return 11pm - date.time
# #                 else:
# #                     return 11pm + (12pm - date.time)
# #             else:
# #                 if date.time < 5pm:
# #                     return 5pm - date.time
# #                 else:
# #                     return 5pm + (12pm - date.time)
# #         elif date == BDay:
                
                
            
in_month = 4
in_date = 29
in_hour = 7
date = pd.datetime(2017, in_month, in_date, in_hour)

# last_bdays_months = get_last_bdays_months()
# get_end_of_day_seconds_left(date, last_bdays_months)